# Sơ lược về Mạng (Network - Basic) 

Môi trường `Mạng` luôn và sẽ là đấu trường với một `hacker`.  Kẻ tấn công có thể làm hầu hết mọi thứ với một truy cập mạng đơn giản, chẳng hạn như quét tìm máy chủ, đưa gói tin, đánh hơi dữ liệu và khai thác máy chủ từ xa.  Nhưng nếu bạn đã tìm cách đi sâu vào, bạn có thể gặp phải một câu hỏi: bạn không có công cụ nào để thực hiện các cuộc tấn công mạng.  Không có netcat.  Không có Wireshark.  Không có trình biên dịch và không có cách nào để cài đặt một trình biên dịch.  Tuy nhiên, bạn có thể ngạc nhiên khi thấy rằng trong nhiều trường hợp, bạn sẽ có một bản cài đặt Python.  Vì vậy, đó là nơi chúng ta sẽ bắt đầu.

Chương này sẽ cung cấp cho bạn một số kiến thức cơ bản về mạng Python bằng cách sử dụng mô-đun `socket`.  Trong quá trình này, chúng tôi sẽ xây dựng máy khách, máy chủ và proxy TCP.  Sau đó, chúng tôi sẽ biến chúng thành netcat của riêng mình, hoàn chỉnh với một trình bao lệnh (shell).  Chương này là nền tảng cho các chương tiếp theo, trong đó chúng tôi sẽ xây dựng một công cụ khám phá máy chủ lưu trữ, triển khai trình dò tìm đa nền tảng và tạo một trojan có thể điều khiển từ xa.

## Mạng trong Python
Các lập trình viên thường có một số công cụ của bên thứ ba để tạo máy chủ và máy khách bằng Python, nhưng mô-đun cốt lõi cho tất cả các công cụ đó là `socket`.  Mô-đun này cho thấy tất cả các phần cần thiết để nhanh chóng viết các máy khách và máy chủ Giao thức Điều khiển Truyền (TCP : Transmission Control Protocol) và Giao thức Dữ liệu Người dùng (UDP: User Datagram Protocol), v.v.  Với mục đích truy cập hoặc duy trì quyền truy cập vào các máy mục tiêu, mô-đun này là tất cả những gì bạn thực sự cần.  Hãy bắt đầu bằng cách tạo một số máy khách và máy chủ đơn giản — hai tập lệnh mạng nhanh phổ biến nhất mà bạn sẽ viết. 



## Máy khách TCP (Transmission Control Protocol)
Rất nhiều lần trong quá trình kiểm tra thâm nhập, chúng tôi (các tác giả) đã cần phải sử dụng máy khách TCP để kiểm tra các dịch vụ, gửi dữ liệu rác, làm mờ hoặc thực hiện bất kỳ tác vụ nào khác. Nếu bạn đang làm việc trong giới hạn của môi trường doanh nghiệp, bạn sẽ không có điều kiện sử dụng các công cụ mạng hoặc trình biên dịch, và đôi khi bạn thậm chí sẽ thiếu những điều cơ bản tuyệt đối, như copy/paste hoặc kết nối với `Internet` . Đây là nơi có thể nhanh chóng tạo một máy khách TCP cực kỳ tiện dụng. Nhưng đừng lo lắng — hãy bắt đầu code.

In [1]:
# Đây là một ứng dụng TCP đơn giản : thực hiện request tới `www.google.com` và in ra dữ liệu nhận được.
import socket

target_host = "www.google.com"
target_port = 80

# tạo một đối tượng socket với 2 params là AF_INET và SOCK_STREAM
# AF_INET cho biết sẽ sử dụng địa chỉ (address) hoặc tên máy chủ(hostname) IPv4 tiêu chuẩn
# SOCK_STREAM cho biết rằng đây sẽ là ứng dụng khách TCP

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# kết nối tới máy khách
client.connect((target_host, target_port))
# thực hiện gửi dữ liệu
client.send(b"GET / HTTP/1.1\r\nHost: google.com\r\n\r\n")
# nhận về dữ liệu 
response = client.recv(4096)
# in ra dữ liệu nhận được 
print("RESPONSE DATA: \n",response.decode())
# đóng cổng kết nối.
client.close()


RESPONSE DATA: 
 HTTP/1.1 301 Moved Permanently
Location: http://www.google.com/
Content-Type: text/html; charset=UTF-8
Date: Fri, 19 Mar 2021 16:48:31 GMT
Expires: Sun, 18 Apr 2021 16:48:31 GMT
Cache-Control: public, max-age=2592000
Server: gws
Content-Length: 219
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN

<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">
<TITLE>301 Moved</TITLE></HEAD><BODY>
<H1>301 Moved</H1>
The document has moved
<A HREF="http://www.google.com/">here</A>.
</BODY></HTML>



Đây là máy khách TCP đơn giản nhất, nhưng đó sẽ là thứ mà bạn sẽ viết thường xuyên nhất.
Đoạn mã này đưa ra một số giả định nghiêm trọng về  `sockets` mà bạn chắc chắn muốn biết. 
- Đầu tiên là kết nối của sẽ luôn `thành công`
- Thứ hai là máy chủ sẽ nhận dữ liệu từ bạn trước (một số máy chủ dự kiến sẽ gửi dữ liệu cho bạn trước và chờ phản hồi của bạn).
- Thứ ba là máy chủ sẽ luôn trả về dữ liệu cho một cách kịp thời

--confused 11--
Mặc dù các lập trình viên có nhiều cách để giải quyết với việc chặn, xử lý ngoại lệ với `socket`, Nhưng đối với một Pentesters thì việc này khá là hiếm


---
**Chú thích**

Tìm hiểu tài liệu về `socket` tại [Đây](https://docs.python.org/3/library/socket.html)

---

## Máy khách UDP (User Datagram Protocol)

Máy khách UDP không khác nhiều so với ứng dụng khách TCP. Cần thực hiện hai thay đổi nhỏ để đưa nó gửi các gói ở dạng UDP.

In [ ]:
import socket

target_host = "www.google.com"
target_port = 80

# tạo đối tượng socket với SOCK_DGRAM
client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# gửi dữ liệu đến host
client.sendto(b"AAABBBCCC",(target_host,target_port))

# nhận dữ liệu trả về 
data, addr = client.recvfrom(4096)

print(data and "data" or "no thing data")
client.close()